In [1]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/Fig_config_utilities.py'

<class 'numpy.ndarray'> (3187, 16101)
<class 'numpy.ndarray'> (3187, 16101)


In [2]:
from shap_model_comparison import SHAPModelComparator

# loading testing set 
subsetted_x_test_centered = x_test_centered.sample(n=1000, random_state=42)
print(f"Using {len(subsetted_x_test_centered)} samples for analysis")

# loading feature names
feature_names = subsetted_x_test_centered.columns.tolist()


ModuleNotFoundError: No module named 'shap_model_comparison'

In [ ]:
# fixing compatibility issue between version 2.0+ XGBoost class and SHAP package

import json

def fix_xgboost_for_shap(model):
    try:
        booster = model.get_booster() if hasattr(model, 'get_booster') else model
        config = json.loads(booster.save_config())
        base_score = config['learner']['learner_model_param']['base_score']
        if base_score.startswith('[') and base_score.endswith(']'):
            base_score_float = float(base_score.strip('[]'))
            config['learner']['learner_model_param']['base_score'] = str(base_score_float)
            booster.load_config(json.dumps(config))
            print("✓ Fixed XGBoost model for SHAP compatibility")
    except Exception as e:
        print(f"Note: XGBoost fix not needed or failed: {e}")
    return model

xgbrf_loaded = fix_xgboost_for_shap(xgbrf_loaded)

In [ ]:
# loading trained models
loaded_models = {'MLR': mlr_loaded, 'XGBRF': xgbrf_loaded} , # add when RNN retrained 'LEMBAS-RNN': rnn 06/01/26

In [ ]:
# for when RNN is retrained and needs to be included 

import torch

class PyTorchRNNWrapper:
    def __init__(self, model, device='cpu'):
        self.model = model
        self.device = device
        self.model.eval()
    
    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        
        X_tensor = torch.FloatTensor(X).to(self.device)
        
        if len(X_tensor.shape) == 2:
            X_tensor = X_tensor.unsqueeze(1)  # Add sequence dimension
        
        with torch.no_grad():
            output = self.model(X_tensor)
        
        return output.cpu().numpy().flatten()

# Load and wrap
rnn_base_model = torch.load('models/lembas_rnn.pth')
rnn_model = PyTorchRNNWrapper(rnn_base_model)

# Test
test_pred = rnn_model.predict(X_analysis[:5])
print(f"✓ RNN loaded and wrapped. Test predictions: {test_pred[:3]}")

In [ ]:
# Test all models
print("\nTesting all models on same 5 samples:")
print("-" * 60)
test_sample = subsetted_x_test_centered.iloc[:5]

for name, model in loaded_models.items():
    try:
        pred = model.predict(test_sample)
        print(f"{name:15s} → predictions: {pred[:3]}")
    except Exception as e:
        print(f"{name:15s} → ERROR: {e}")
        print("  ⚠️  Fix this before proceeding!")